In [1]:
import torch
import torch.nn as nn
import time
import BiLSTM_CRF
from utils import *
from NER_dataset import *
from tag_mapping import *
from word_mapping import *
from tqdm import tqdm
from check import *

In [2]:

def BiLSTM_CRF_check(language):
    train_data, valid_data = get_data_set(language)
    TagMapping = tag_mapping(language)
    WordMapping = word_mapping(train_data)

    train_data.get_tag_mapping(TagMapping.encode_mapping)
    train_data.get_word_mapping(WordMapping.encode_mapping)
    valid_data.get_tag_mapping(TagMapping.encode_mapping)
    valid_data.get_word_mapping(WordMapping.encode_mapping)

    tag_size = TagMapping.num_tag
    vocab_size = WordMapping.num_word

    dropout = 0.5
    embed_size = 256
    hidden_size = 256
    batch_size = 32
    max_epoch = 20
    lr = 0.001
    clip_max_norm = 5.0
    model_save_path = f'./model/{language}model.pth'

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = BiLSTM_CRF.BiLSTMCRF(tag_size, vocab_size, dropout, embed_size, hidden_size).to(device)
    for name, param in model.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, 0, 0.01)
        else:
            nn.init.constant_(param.data, 0)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    print('start training...')
    for epoch in range(max_epoch):
        num_iter = 0
        for sentences, tags in tqdm(batch_iter(train_data, batch_size=batch_size)):
            num_iter += 1
            sentences, sent_lengths = pad(sentences, vocab_size - 1, device)
            tags, _ = pad(tags, tag_size - 1, device)

            optimizer.zero_grad()
            batch_loss = model(sentences, tags, sent_lengths)  # shape: (b,)
            loss = batch_loss.mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_max_norm)
            optimizer.step()

            if num_iter % 100 == 0:
                print('Epoch: %d, Iter: %d, Loss: %.4f' % (epoch, num_iter, loss.item()))

    model.save(model_save_path)
    model = BiLSTM_CRF.BiLSTMCRF.load(model_save_path, device)
    print('start testing...')

    my_path = f'my_{language}_BiLSTM_CRF_result.txt'
    result_file = open(my_path, 'w')
    model.eval()
    with torch.no_grad():
        for sentences, tags in valid_data:
            padded_sentences, sent_lengths = pad([sentences], vocab_size - 1, device)
            pred_tags = model.predict(padded_sentences, sent_lengths)
            sent = WordMapping.decode(sentences)
            pred_tags = TagMapping.decode(pred_tags[0])
            for token, pred_tag in zip(sent, pred_tags):
                result_file.write(f'{token} {pred_tag}\n')
            result_file.write('\n')

    result_file.close()

    gold_path = f"./NER/{language}/validation.txt"
    check(language, gold_path, my_path)


In [3]:
print('---------------- Chinese Result ------------------')
BiLSTM_CRF_check('Chinese')
print('---------------- English Result ------------------')
BiLSTM_CRF_check('English')

---------------- Chinese Result ------------------
start training...


0it [00:00, ?it/s]

tags.shape =  torch.Size([32, 176])
emit_score.shape =  torch.Size([32, 176, 34])
tags.unsqueeze(dim=2).shape =  torch.Size([32, 176, 1])
real_score.shape =  torch.Size([32, 176])
temp.shape =  torch.Size([32, 1, 34])
mask.shape =  torch.Size([32, 176])
num_unfinished.shape =  torch.Size([])
num_unfinished =  tensor(32, device='cuda:0')
temp_unfinished.shape =  torch.Size([32, 1, 34])
emit_plus_transition.shape =  torch.Size([32, 34, 34])
forward_score.shape =  torch.Size([32, 34, 34])
max_score.shape =  torch.Size([32, 1, 34])
temp_unfinished.shape =  torch.Size([32, 1, 34])
temp.shape =  torch.Size([32, 1, 34])
num_unfinished.shape =  torch.Size([])
num_unfinished =  tensor(32, device='cuda:0')
temp_unfinished.shape =  torch.Size([32, 1, 34])
emit_plus_transition.shape =  torch.Size([32, 34, 34])
forward_score.shape =  torch.Size([32, 34, 34])
max_score.shape =  torch.Size([32, 1, 34])
temp_unfinished.shape =  torch.Size([32, 1, 34])
temp.shape =  torch.Size([32, 1, 34])
num_unfinish

SystemExit: 0